In [1]:
from rerank import BGEReranker
from search_with_bge import QdrantSearchBGE
import pandas as pd
from tqdm import tqdm
from typing import List, Dict, Callable


c:\Users\Admin\miniconda3\envs\crypto_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def fix_cid_list(x):
    try:
        return [int(i) for i in x.strip("[]").split()]
    except:
        return []

In [3]:
bge_search_instance = QdrantSearchBGE(
        host="http://localhost:6333",
        collection_name= "law_corpus_bge_v3",
        model_name="BAAI/bge-m3",
        use_fp16=True
    )

Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]


In [4]:
# client = bge_search_instance.client
# query_test = "Hồ sơ tuyển chọn Thẩm phán Tòa án nhân dân cấp huyện phải được giao lại cho Chánh án Tòa án nhân dân cấp tỉnh quản lý khi nào?"

In [5]:

# client.search(
#             collection_name=bge_search_instance.collection_name,
#             query_vector= bge_search_instance.encode_query(query_test),
#             limit=30,
#             with_payload=True
#         )

In [6]:
# bge_results = bge_search_instance.search("Thông tư này hướng dẫn tuần tra, canh gác về Đê Điều ",limit=10)
# # doc_search = [doc.payload['text'] for doc in bge_results]
# bge_results

In [7]:
model_funcs = {
    "BGE-m3": lambda q: bge_search_instance.search(q, limit=50)
}

In [8]:
def evaluate_recall_at_k(
    queries_df: pd.DataFrame,
    ground_truth_col: str,  # ví dụ: "cid"
    model_funcs: Dict,
    query_col: str = "question",
    k_list: List[int] = [3, 5, 10]
) -> pd.DataFrame:
    """
    Đánh giá Recall@K cho nhiều mô hình truy hồi.

    Parameters:
    - queries_df: dataframe chứa câu hỏi và ground truth
    - ground_truth_col: tên cột chứa list cid đúng (dạng string list)
    - query_col: tên cột chứa câu hỏi
    - model_funcs: dict chứa {"tên mô hình": func_truy_hồi(query) → list cid}
    - k_list: danh sách K để tính recall

    Returns:
    - DataFrame với các dòng là mô hình và cột là R@k
    """
    recall_results = {model_name: {f"R@{k}": 0 for k in k_list} for model_name in model_funcs}
    total = len(queries_df)

    for row in tqdm(queries_df.itertuples(), total=total):
        query = getattr(row, query_col)
        gt_cids = set(getattr(row, ground_truth_col))  # ví dụ: "[1, 2]"
        for model_name, retrieve_func in model_funcs.items():
            # print(f"Evaluating model: {model_name} for query: {query}")
            retrieved_docs = retrieve_func(query)  # list[dict], có key "cid"
            # print(f"Model: {model_name}, Query: {query}, Ground Truth CIDs: {gt_cids}")
            # for doc in retrieved_docs[:5]:
            #     print("cid:", doc['cid'] , "score", doc['score'], "text:", doc['text'][:100])
            predicted_cids = [doc["cid"] for doc in retrieved_docs]
            for k in k_list:
                top_k = set(predicted_cids[:k])
                if gt_cids & top_k:  # có giao là đúng
                    recall_results[model_name][f"R@{k}"] += 1

    # Chuyển sang % và DataFrame
    for model_name in recall_results:
        for k in k_list:
            recall_results[model_name][f"R@{k}"] = round(
                recall_results[model_name][f"R@{k}"] / total * 100, 2
            )

    return pd.DataFrame.from_dict(recall_results, orient="index")

In [9]:
train_df = pd.read_csv(r"D:\Data\Legal-Retrieval\data\train.csv")
train_df = train_df[:1000]
train_df["cid"] = train_df["cid"].apply(fix_cid_list)

recall_df = evaluate_recall_at_k(
    queries_df=train_df,
    ground_truth_col="cid",  # giả sử đây là chuỗi dạng "[1, 2, 3]"
    query_col="question",
    model_funcs=model_funcs,
    k_list=[3,5,10,30]
)
print(recall_df)
# train_df

100%|██████████| 1000/1000 [04:36<00:00,  3.62it/s]

         R@3   R@5  R@10  R@30
BGE-m3  58.4  65.6  74.4  84.2


In [10]:
ccx

NameError: name 'ccx' is not defined

In [ ]:
reranker_instance = BGEReranker(model_name="BAAI/bge-reranker-v2-m3",use_fp16=True)
reranked_results = reranker_instance.rerank(query="Thông tư này hướng dẫn tuần tra, canh gác về Đê Điều ", documents=doc_search, topk=5)
reranked_results

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.',
 '1. Người tuần tra, canh gác đê trong khi làm nhiệm vụ phát hiện thấy \xa0có hư hỏng của đê Điều phải tìm mọi cách nhanh chóng báo cáo cán bộ chuyên trách quản lý đê Điều và Ban Chỉ huy phòng, chống lụt, bão xã để tiến hành xử lý kịp thời.\n2. Nội dung báo cáo:\n- Thời gian phát hiện hư hỏng;\n- Vị trí, đặc điểm, kích thước, diễn biến của hư hỏng và mức độ nguy hiểm;\n- Đề xuất biện pháp xử lý.\n3. Trường hợp xét thấy hư hỏng có khả năng diễn biến xấu, đội trưởng phải cử người tăng cường, theo dõi tại chỗ và cứ 30 phút phải báo cáo một lần.\nTrường hợp hư hỏng có nguy cơ đe dọa an toàn của công trình, phải tiến hành xử lý gấp nhằm ngăn chặn và hạn chế hư hỏng phát triển thêm đồng thời phát tín hiệu báo động theo quy định khoản 2 Điều 7 của Thông tư này. Trong khi chờ lực lượng ứng cứu, những người được phân công theo dõ